In [2]:
from domain_adaptation.datasets import SwaVDataset
from domain_adaptation.archs import swav
from domain_adaptation.models import resnet
import pathlib
import os
import matplotlib.pyplot as plt

import tensorflow as tf

In [1]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # resize the image to the desired size
  return tf.image.resize(img, [180, 180])

def process_path(file_path):
    label = tf.strings.split(file_path, os.sep)[-2]
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img

# labeled_ds = ds.shuffle(1024).map(process_path)

In [3]:
flowers_root = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)
flowers_root = pathlib.Path(flowers_root)

In [4]:
flowers_ds = tf.data.Dataset.list_files(str(flowers_root/'*/*'))
flowers_ds = flowers_ds.map(process_path)

In [5]:
a = SwaVDataset.SwaVDataset(flowers_ds, 
                            override=False, 
                            nmb_crops=[3, 4],
                            size_crops=[224, 168],
                            min_scale_crops=[0.14, 0.16],
                            max_scale_crops=[1., 1.])

In [6]:
ds = a.dataset_swaved

In [7]:
model = resnet.Resnet50().model
swav_mod = swav.SwAV(model=model)

In [8]:
swav_mod.prototype_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2048)]            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_53 (Batc (None, 1024)              4096      
_________________________________________________________________
activation_49 (Activation)   (None, 1024)              0         
_________________________________________________________________
projection (Dense)           (None, 96)                98400     
_________________________________________________________________
tf.math.l2_normalize (TFOpLa (None, 96)                0         
_________________________________________________________________
prototype (Dense)            (None, 10)                960   

In [9]:
decay_steps = 1000
lr_decayed_fn = tf.keras.experimental.CosineDecay(initial_learning_rate=0.1, 
                                                decay_steps=decay_steps)
opt = tf.keras.optimizers.SGD(learning_rate=lr_decayed_fn)

hist = swav_mod.fit(ds, optimizer=opt, epochs=10)

0it [00:28, ?it/s]


AttributeError: 'tuple' object has no attribute 'tolist'